In [10]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [12]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [13]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [14]:
# Add the latitudes and longitudes to a list.
# Can only unzip a zipped tuple once before it is removed from the computer's memory. 
coordinates = list(lat_lngs)

In [15]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

33.06203940432884 125.9456524049699
-42.00406658433654 -83.11067126415524
-87.3292324285441 117.62999518522514
-59.25277776771647 127.91988875149599
-67.33657503048036 177.232668034264
-64.93878363953029 66.73998466447804
48.64785967996272 28.247002342760425
-67.84261552519538 -127.04998722479078
26.92313033044759 146.45037012930504
31.31064652976049 153.22820320568184
31.74412626293386 120.0819874423446
68.91801921208935 -0.18819229607905186
67.32966313268201 53.9103462660195
10.44926684205015 -48.36528866089148
-87.11171581961898 -96.77079420475071
23.226680458302383 -160.9087263277363
10.521052085884946 176.9716998180611
-76.75868381444431 67.98678200291488
70.73686991591507 -109.97337866599368
16.82569841178146 -124.05473882893243
36.28409912135821 60.28985503564249
-77.3233555981003 82.01770354368819
-37.68539782012834 9.873395614057756
84.44903614419258 -104.49739357703062
15.783214881971375 -146.93682103365964
-80.38292215798761 59.798712876934076
-51.58649096538984 108.34184434

-43.21849689843695 -33.52143228110242
-81.6160915367212 -99.13790244699636
17.161191621647788 74.91285883220945
21.19064315782751 -3.8908830420898255
76.29960195877516 -102.7724368898143
-70.1233962341714 169.34996913606284
88.95368728163999 35.70180655414211
-42.90615474029461 175.3601106335803
-81.02878361550285 -109.10268636877757
-67.5255278934547 38.0132172704688
68.9627676462043 -176.17250180562
-64.13109801172517 -58.060524188447715
-13.892332129785117 -141.25358550176395
-39.02416145859767 -178.87777468941786
28.59385863117423 -49.793819208405296
-57.16874177506921 -13.917646662894668
-31.787123322859976 -21.732563645689453
-85.89904289195053 -37.20259595773496
-77.012044982886 24.050734314540904
87.0722652428793 112.67859823164866
79.38528487354287 132.44938434193125
-24.733639985519275 22.17576136204275
35.16135754349939 17.79339689503729
-62.7655703174715 136.27736991573374
-60.22601743374146 58.55588169079118
33.61114440309635 -158.0819110162023
-78.96759617814809 -25.82564

In [16]:
from citipy import citipy

In [17]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

741

In [18]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [19]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# use the enumerate() method to get the index of the city for logging purposes and the city for creating an endpoint URL
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
# Add a try-except block to prevent API request from stopping prematurely
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_desc = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_desc})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    #Generally, it isn't good coding practice to add the pass statement to the except block. Ideally, we want to handle or catch each error as it happens and do something specific (e.g., add another try block or print out the error).

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | fukue
Processing Record 2 of Set 1 | ancud
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | portland
Processing Record 5 of Set 1 | kaitangata
Processing Record 6 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 7 of Set 1 | dzhuryn
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | katsuura
Processing Record 10 of Set 1 | hasaki
Processing Record 11 of Set 1 | changzhou
Processing Record 12 of Set 1 | brae
Processing Record 13 of Set 1 | iskateley
Processing Record 14 of Set 1 | cayenne
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | butaritari
Processing Record 18 of Set 1 | yellowknife
Processing Record 19 of Set 1 | constitucion
Processing Record 20 of Set 1 | mashhad
Processing Record 21 of Set 1 | busselton
Processing Record 22 of Set 1 | saldanha
Processing Record 23 

Processing Record 37 of Set 4 | port blair
Processing Record 38 of Set 4 | matara
Processing Record 39 of Set 4 | cotonou
Processing Record 40 of Set 4 | victoria
Processing Record 41 of Set 4 | kavieng
Processing Record 42 of Set 4 | coquimbo
Processing Record 43 of Set 4 | haines junction
Processing Record 44 of Set 4 | aykhal
Processing Record 45 of Set 4 | port alfred
Processing Record 46 of Set 4 | charlestown
Processing Record 47 of Set 4 | goundi
Processing Record 48 of Set 4 | kualakapuas
Processing Record 49 of Set 4 | kamenskoye
City not found. Skipping...
Processing Record 50 of Set 4 | fortuna
Processing Record 1 of Set 5 | xuanwu
Processing Record 2 of Set 5 | terney
Processing Record 3 of Set 5 | kelapa sawit
Processing Record 4 of Set 5 | samalaeulu
City not found. Skipping...
Processing Record 5 of Set 5 | vrangel
Processing Record 6 of Set 5 | hambantota
Processing Record 7 of Set 5 | katsiveli
City not found. Skipping...
Processing Record 8 of Set 5 | areosa
Processin

Processing Record 25 of Set 8 | miraflores
Processing Record 26 of Set 8 | san andres
Processing Record 27 of Set 8 | aksu
Processing Record 28 of Set 8 | kimberley
Processing Record 29 of Set 8 | berlevag
Processing Record 30 of Set 8 | clyde river
Processing Record 31 of Set 8 | sentyabrskiy
City not found. Skipping...
Processing Record 32 of Set 8 | gangapur
Processing Record 33 of Set 8 | anori
Processing Record 34 of Set 8 | chicago
Processing Record 35 of Set 8 | labytnangi
Processing Record 36 of Set 8 | tongliao
Processing Record 37 of Set 8 | morondava
Processing Record 38 of Set 8 | grootfontein
Processing Record 39 of Set 8 | bilma
Processing Record 40 of Set 8 | asau
Processing Record 41 of Set 8 | yichun
Processing Record 42 of Set 8 | taltal
Processing Record 43 of Set 8 | chebsara
Processing Record 44 of Set 8 | netanya
Processing Record 45 of Set 8 | naberera
Processing Record 46 of Set 8 | el balyana
City not found. Skipping...
Processing Record 47 of Set 8 | juegang
P

Processing Record 12 of Set 12 | bambanglipuro
Processing Record 13 of Set 12 | male
Processing Record 14 of Set 12 | listowel
Processing Record 15 of Set 12 | le vauclin
Processing Record 16 of Set 12 | bentiu
Processing Record 17 of Set 12 | tombouctou
Processing Record 18 of Set 12 | timizart
Processing Record 19 of Set 12 | shenjiamen
Processing Record 20 of Set 12 | shahr-e babak
Processing Record 21 of Set 12 | hutchinson
Processing Record 22 of Set 12 | natal
Processing Record 23 of Set 12 | dzhusaly
City not found. Skipping...
Processing Record 24 of Set 12 | savannah bight
Processing Record 25 of Set 12 | dieppe
Processing Record 26 of Set 12 | port augusta
Processing Record 27 of Set 12 | ukiah
Processing Record 28 of Set 12 | cordoba
Processing Record 29 of Set 12 | hukuntsi
Processing Record 30 of Set 12 | frontera
Processing Record 31 of Set 12 | curuca
Processing Record 32 of Set 12 | singaraja
Processing Record 33 of Set 12 | sisimiut
Processing Record 34 of Set 12 | wan

In [22]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [21]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Fukue,JP,32.6881,128.8419,63.57,78,94,4.54,overcast clouds
1,Ancud,CL,-41.8697,-73.8203,52.84,84,94,19.33,light rain
2,Albany,US,42.6001,-73.9662,79.74,61,97,7.00,overcast clouds
3,Portland,US,45.5234,-122.6762,57.99,66,99,1.99,overcast clouds
4,Kaitangata,NZ,-46.2817,169.8464,44.83,82,36,18.52,scattered clouds
5,Dzhuryn,UA,49.0329,25.5839,59.49,73,9,8.39,clear sky
6,Rikitea,PF,-23.1203,-134.9692,72.63,78,100,0.74,overcast clouds
7,Katsuura,JP,35.1333,140.3000,66.67,80,100,11.54,light rain
8,Hasaki,JP,35.7333,140.8333,64.80,77,100,12.71,overcast clouds
9,Changzhou,CN,31.7833,119.9667,63.59,87,91,2.89,overcast clouds


In [25]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")